# 利用Google Colab下载文件到谷歌云端硬盘




**什么是Google Colab?**
+ Colaboratory 是一个 Google 研究项目，旨在帮助传播机器学习培训和研究成果。它是一个 Jupyter 笔记本环境，不需要进行任何设置就可以使用，并且完全在云端运行。
+ Colaboratory 笔记本存储在 Google 云端硬盘中，并且可以共享，就如同您使用 Google 文档或表格一样。Colaboratory 可免费使用。
+ 利用Colaboratory ，可以方便的使用Keras,TensorFlow,PyTorch,OpenCV等框架进行深度学习应用的开发。

# 准备及说明


In [0]:
#@title 第一步，运行以挂载谷歌云端硬盘
from google.colab import drive
drive.mount('/content/drive')

只支持直链或种子下载，请获得直链或种子后再使用

使用了最简单的wget与aria2指令进行直链下载，大佬勿喷

种子下载部分源码在[Github](https://github.com/r12habh/Torrent-To-Google-Drive-Downloader-v2)获得，只是做了整合，后又发现aria2也可实现种子下载功能，已添加

如需下载文件到OneDrive，请自行使用rclone挂载，以后也许会更新



**挂载谷歌云端硬盘时，由于bug存在，无法写入根目录。如需往盘内下载文件或移动文件至子目录，请先在盘内新建子目录，下载时填写子目录**

# 直链下载

In [0]:
#@title 1.下载单文件
#@markdown <p>输入直链及下载位置</p>
#@markdown <p>默认位置为/content</p>
links = "" #@param {type:"string"}
savepath = "/content" #@param {type:"string"}
!wget -c "$links" -P "$savepath"

In [0]:
#@title 2.批量下载(wget)
#@markdown <p>输入下载位置</p>
#@markdown <p>在左侧新建文件"filelist.txt",双击输入直链，一行一个</p>
savepath = "/content" #@param {type:"string"}
!wget -i filelist.txt -P "$savepath"

In [0]:
#@title 2.批量下载(aria2)
#@markdown <p>输入下载文件保存位置</p>
#@markdown <p>在左侧新建文件"filelist.txt",双击输入直链，一行一个</p>
保存位置 = "/content" #@param {type:"string"}
线程数 = "5" #@param {type:"string"}
同时下载文件数 = "50" #@param {type:"string"}
!sudo apt-get install aria2
!aria2c -c -x"$线程数" -j"$同时下载文件数" -ifilelist.txt -d"$保存位置"

# 种子下载

In [0]:
#@title 1.安装aria2与libtorrent
!sudo apt-get install aria2
!apt install python3-libtorrent

import libtorrent as lt

ses = lt.session()
ses.listen_on(6881, 6891)
downloads = []

In [0]:
#@title 2.1使用种子文件
#@markdown <p>点击运行后上传种子文件(可传多个）并输入下载路径</p>
#@markdown <p>路径默认输/content</p>
from google.colab import files
source = files.upload()
params = {
    "save_path": input("输入下载路径："),
    "ti": lt.torrent_info(list(source.keys())[0]),
}
downloads.append(ses.add_torrent(params))

In [0]:
#@title 2.2使用磁力链接
#@markdown <p>点击运行后输入下载路径与磁力链接</p>
#@markdown <p>路径默认输/content</p>
params = {"save_path": input("输入下载路径")}

while True:
    magnet_link = input("输入磁力链接或者输入Exit: ")
    if magnet_link.lower() == "exit":
        break
    downloads.append(
        lt.add_magnet_uri(ses, magnet_link, params)
    )

In [0]:
#@title 3.1使用libtorrent下载
import time
from IPython.display import display
import ipywidgets as widgets

state_str = [
    "queued",
    "checking",
    "downloading metadata",
    "downloading",
    "finished",
    "seeding",
    "allocating",
    "checking fastresume",
]

layout = widgets.Layout(width="auto")
style = {"description_width": "initial"}
download_bars = [
    widgets.FloatSlider(
        step=0.01, disabled=True, layout=layout, style=style
    )
    for _ in downloads
]
display(*download_bars)

while downloads:
    next_shift = 0
    for index, download in enumerate(downloads[:]):
        bar = download_bars[index + next_shift]
        if not download.is_seed():
            s = download.status()

            bar.description = " ".join(
                [
                    download.name(),
                    str(s.download_rate / 1000),
                    "kB/s",
                    state_str[s.state],
                ]
            )
            bar.value = s.progress * 100
        else:
            next_shift -= 1
            ses.remove_torrent(download)
            downloads.remove(download)
            bar.close() # Seems to be not working in Colab (see https://github.com/googlecolab/colabtools/issues/726#issue-486731758)
            download_bars.remove(bar)
            print(download.name(), "complete")
    time.sleep(1)

In [0]:
#@title 3.2使用aria2下载
#@markdown <p>种子可以是磁力链接也可以是种子文件</p>
#@markdown <p>种子文件先在2.1上传，然后在左侧复制文件路径填入</p>
种子 = "magnet:?xt=urn:btih:B6C323FCAE96F3AF40AE695662717A6BB9BE665F" #@param {type:"string"}
保存位置 = "/content" #@param {type:"string"}
!aria2c -c "$种子" -d"$保存位置"

# 压缩与解压

In [0]:
#@markdown <p>输入压缩包文件名与解压目录</p>
#@markdown <p>使用7z解压缩</p>
filename = "/content/test.zip" #@param {type:"string"}
pathto = "/content" #@param {type:"string"}
!7z  x "$filename" -o/"$pathto"

In [0]:
#@markdown <p>输入压缩包密码和文件名和解压目录</p>
#@markdown <p>使用7z解压缩</p>
password = "" #@param {type:"string"}
filename = "filename.zip" #@param {type:"string"}
pathto = "/content" #@param {type:"string"}
!7z  x  -p"$password" "$filename" -o/"$pathto"

In [0]:
#@markdown <p>输入要压缩的文件/文件夹路径、保存的压缩包名称和压缩包位置</p>
#@markdown <p>使用7z压缩</p>
path = "/content/test.txt" #@param {type:"string"}
filename = "yasuo.zip" #@param {type:"string"}
pathto = "/content" #@param {type:"string"}
!7z a -tzip -r "$filename" "$path" -o/"$pathto"